# Tutorial 2 - Classification (Test)

## Open Dataset : MURA Dataset (Ver 1.0)

- MUsculoskeletal RAdiographs (Bone X-rays)         
- https://stanfordmlgroup.github.io/competitions/mura/
- https://arxiv.org/abs/1712.06957

In [1]:
# Python 3.6
from __future__ import absolute_import, division, print_function
import os
import numpy as np
import h5py


import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import skimage
import scipy.io as sio

import keras
from keras.layers import Input

import keras.applications.inception_v3 as inception_v3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.callbacks import  ModelCheckpoint, CSVLogger, ReduceLROnPlateau
import tensorflow as tf

from utils.custom_generator import custom_image_generator


Using TensorFlow backend.


## Parameter Setting

In [3]:
########################################################################
########################################################################
save_date = '1022'
# num_classes = 7 ## 14
########################################################################
########################################################################


*Network Input Size: 64x64x64


## Path Setting

In [2]:
datapath = '../Dataset/MURA-v1.0/h5py/batch/'


save_path = os.path.join(os.getcwd(), 'Result/saved_model')
if not os.path.isdir(save_path):
    os.makedirs(save_path)    
model_path = os.path.join(save_path, model_name)

tf_graph_path = './Result/tf_graph/'    
if not os.path.isdir(tf_graph_path):
    os.makedirs(tf_graph_path)   
    
graph_path = './Result/graph_log/'    
if not os.path.isdir(graph_path):
    os.makedirs(graph_path)   

## Load Test Dataset

In [4]:
extraval = h5py.File(datapath +'DataExtra.h5', 'r')
extimg = extraval['imgsExtra']
X_test = np.array(extimg)
extlabel = extraval['labelExtra']
y_test = np.array(extlabel)

print(np.shape(X_test))
print(np.shape(y_test))
print("* Test Number of Datasets: " + str(len(X_test)))

(505, 64, 64, 64, 1)
(505, 2)
* Test Number of Datasets: 505


## Load Trained Model

In [6]:
#####################################################################################
#####################################################################################
trained_model = load_model('../Result/saved_model/MURA_Inception_v3_1022_batch1.h5')
#####################################################################################
#####################################################################################

## Prediction

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
    
print("*** Predictction ***")
y_pred_temp = trained_model.predict(X_test, verbose=0)

y_pred = []
y_pred_prob0 =[]
y_pred_prob1 =[]
y_pred_prob2 =[]
y_pred_prob3 =[]
y_pred_prob4 =[]
y_pred_prob5 =[]
y_pred_prob6 =[]


for i in range(len(y_pred_temp)):
    y_pred.append(np.argmax(y_pred_temp[i], axis=None, out=None))
    y_pred_prob0.append(round(y_pred_temp[i,0], 3))
    y_pred_prob1.append(round(y_pred_temp[i,1], 3))
    y_pred_prob2.append(round(y_pred_temp[i,2], 3))
    y_pred_prob3.append(round(y_pred_temp[i,3], 3))
    y_pred_prob4.append(round(y_pred_temp[i,4], 3))
    y_pred_prob5.append(round(y_pred_temp[i,5], 3))
    y_pred_prob6.append(round(y_pred_temp[i,6], 3))


### One-hot decoding
y_test_mod = []
for i in range(len(y_test)):
    if y_test[i][0] == 0:
        val = 1
        y_test_mod.append(val)
    else:
        val = 0
        y_test_mod.append(val)

print("\n***** Prediction Complete *****")

-- Predict --


## Evaluation

In [ ]:
""" Confusion Matrix """
confusion = metrics.confusion_matrix(y_test_mod, y_pred)
print(confusion)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

""" Accuracy """
print("Accuracy: ", round(((TP + TN)/(TP + TN + FP + FN))*100, 2))

""" Sensitivity """
sensitivity = round((TP/(FN+TP))*100, 2)
print("Sensitivity: ", sensitivity)

""" Specificity """
specificity = round((TN/(TN+FP))*100, 2)
print("Specificity: ", specificity)

""" AUC """
roc_auc = metrics.auc(fpr, tpr)
print("AUC: ", round(roc_auc, 2))

""" PPV """
PPV = round((TP/(TP+FP))*100, 2)
print("PPV: ", PPV)

""" NPV """
NPV = round((TN/(TN+FN))*100, 2)
print("NPV: ", NPV)


## Save Result (.csv)

In [ ]:
###################################################################
##################################################################
lists = [ID, y_pred_prob0, y_pred_prob1, y_pred_prob2, y_pred_prob3, y_pred_prob4, y_pred_prob5, y_pred_prob6, y_pred, y_test_mod]
export_data = zip_longest(*lists, fillvalue = '')
with open('../Result/result_csv/extra_image_result1.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(("ID", "Prob0", "Prob1", "Prob2", "Prob3", "Prob4", "Prob5", "Prob6", "Pred", "GT"))
    writer.writerows(export_data)
###################################################################
###################################################################

## Check Result

In [ ]:
Test_csv_result = pd.read_csv("../Result/result_csv/Batch1.csv")
Test_csv_result.tail(100)